버전관리
=============
* v1.0 
  - (22.11.13. 13:00 신재호)
  - 전체 코드 생성
* v1.1
  - (22.11.14. 13:00 신재호)
  - encoder 객체 생성 위치 변경 (labelencoder, RobustScaler,StandardScaler )


# 1. setting

> ## 1.1 패키지 import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 

> ## 1.2 데이터 READ

In [ ]:
# 구글 코랩에서 데이터 attach를 위한 code
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 구글 드라이브 CSV 파일 디렉터리
FILE_DIR = '/content/drive/MyDrive/Credit score classification.csv'

In [ ]:
#전처리 없는 원본 데이터
raw_df = pd.read_csv(FILE_DIR)

#전처리 하고, 학습을 위한 데이터
train_df = pd.read_csv(FILE_DIR)


# <span style="background-color:#FAF4C0"><span style='color: black'>2. Preprocessing</span></span>

> ## 2.1 feature select
- https://www.notion.so/rphabet/Credit-Score-Classification-Data_1-6891e4b3a6324a2abf8ff2d8740b2b63

In [ ]:
#전처리 이전 모든 Feature
raw_cols =[
       'ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score']

In [ ]:
# 라벨 인코딩 대상 Feature
label_cols = [
    'Customer_ID',
    'Payment_of_Min_Amount',
    'Payment_Behaviour', # 라벨인코딩 또는 순서대로 대치  
]

# 삭제 대상 Feature
drop_cols =[
'ID',
'Name',
'SSN',
'Occupation',
'Annual_Income',
'Type_of_Loan']

# 더미생성을 위한 Featue
dummy_cols = [
    'Num_of_Loan',             # 5 기준
    'Changed_Credit_Limit',    # 12기준
    'Credit_History_Age',      # 180 기준
]

# 로그변환 대상 Feature
log_scale_cols = [
    'Monthly_Inhand_Salary'
]

# sqrt 변환을 위한 Feature
sqrt_scale_cols = [
    'Outstanding_Debt'
]

#정규화를 위한 feature
standard_cols =[
    
]

#원핫인코딩 대상 feature
onehot_cols =[
    
]

#PCA 대상 Feature
pca_cols = [
    
]

# Feature 특징을 고려한 개별 전처리
others_cols =[
    'Age', # 특정 숫자 기준으로 구간화
    'Credit_Mix', # Bad 0, Standard 1 , Good 2
    'Credit_Score', # Poor 0 , Standard 1, Good 2로 대치
]



#전처리 없는 Feature
processless_cols = (set(raw_cols) - set(label_cols) - set(dummy_cols) - set(drop_cols)
                      - set(log_scale_cols) - set(sqrt_scale_cols) - set(others_cols)
                      -set(pca_cols) - set(standard_cols) - set(onehot_cols))
processless_cols = list(processless_cols)

In [ ]:
def print_cols(name,cols):
  print('\n',name)
  for i in cols:
    print('\t\t',i)

print_cols(' 라벨인코딩 : ',label_cols )

print_cols(' 라벨인코딩 : ',label_cols )

print_cols('\n Feature 제거 : ', drop_cols)

print_cols('\n Dummy생성 : ',dummy_cols)

print_cols('\n 로그 변환 : ', log_scale_cols)

print_cols('\n Sqrt 변환 : ', sqrt_scale_cols)

print_cols('\n pca 처리 : ', pca_cols)

print_cols('\n 정규화처리 : ', standard_cols)

print_cols('\n 원핫인코딩',onehot_cols)

print_cols('\n 기타 처리 : ',others_cols)

print_cols('\n 전처리 없음 :',processless_cols)




  라벨인코딩 : 
		 Customer_ID
		 Payment_of_Min_Amount
		 Payment_Behaviour

  라벨인코딩 : 
		 Customer_ID
		 Payment_of_Min_Amount
		 Payment_Behaviour

 
 Feature 제거 : 
		 ID
		 Name
		 SSN
		 Occupation
		 Annual_Income
		 Type_of_Loan

 
 Dummy생성 : 
		 Num_of_Loan
		 Changed_Credit_Limit
		 Credit_History_Age

 
 로그 변환 : 
		 Monthly_Inhand_Salary

 
 Sqrt 변환 : 
		 Outstanding_Debt

 
 pca 처리 : 

 
 정규화처리 : 

 
 원핫인코딩

 
 기타 처리 : 
		 Age
		 Credit_Mix
		 Credit_Score

 
 전처리 없음 :
		 Num_of_Delayed_Payment
		 Delay_from_due_date
		 Amount_invested_monthly
		 Month
		 Interest_Rate
		 Credit_Utilization_Ratio
		 Num_Credit_Inquiries
		 Total_EMI_per_month
		 Monthly_Balance
		 Num_Bank_Accounts
		 Num_Credit_Card


> ## 2.2 Feature Drop

In [ ]:
def drop_do(cols,input_data):
  '''
  feature 삭제를 위한 code
  param:
    cols : 삭제 대상 feature
    input_data : 삭제전 전체 데이터

  return : 전체 데이터에서 삭제 대상 feture 삭제후 결과
  '''
  data = input_data.copy()

  return data.drop(cols,axis=1)

> ## 2.3 라벨인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoding_do(cols,input_data):
    '''
    라벨 인코딩을 위한 함수
    param : 
    cols : 라벨인코딩 대상 feture
    data : 전처리를 위한 데이터

    return : 전체 데이터에서 대상 특징만 라벨인코딩 한 결과
    '''

    data = input_data.copy()

    #라벨인코더 객체 생성
  

    #라벨인코딩 대상 Feature 하나씩 처리
    for col in cols:
      encoder = LabelEncoder()
      data[col] = encoder.fit_transform(data[col])

    return data

> ## 2.4 로그 처리

In [ ]:
def log_transform_do(cols,input_data):
  '''
  로그변환을 위한 코드
  params : 
    cols : 인코딩 대상 features
    input_data : 원본 데이터
  '''
  data = input_data.copy()

  for col in cols:
    data[col] = np.log(data[col])

  return data


> ## 2.5. SQRT 처리

In [ ]:
def sqrt_transform_do(cols,input_data):
  '''
  sqrt 처리를 위한 코드 
  param:
    cols : 인코딩 대상 features
    data : 전처리를 위한 데이터
  '''
  
  data = input_data.copy()
  
  for col in cols:
    data[col] = np.sqrt(data[col])

  return data

> ## 2.6. 더미 생성 

In [ ]:
def dummy_make_do (cols,input_data,thresh=5):
    """
    특정값을 기준으로 Y/N 더미변수로 만들기 위한 함수
    params:
      cols : 전처리 대상 features
      data : 원본 데이터
      thresh : 0/1 으로 만들기 위한 기준
    """

    data = input_data.copy()

    for col in cols:
    data[col] = data.apply(get_dummy,axis=1,args=[col,thresh])
    return data

def get_dummy(df, col ,thresh=5):

    if df[col] >= thresh :
      return 1
    elif df[col] < thresh:
      return 0
    else:
      return 2

> ## 2.7 표준화

In [ ]:
import sklearn
from sklearn.preprocessing import *

def standard_transform_do (cols,input_data):
  '''
  표준화를 위한 함수
  '''

  data = input_data.copy()  #데이터 복사

  for col in cols:
    scaler = StandardScaler()
    data[col] = scaler.fit_transform(data[col])
  return data

def robust_transform_do (cols,input_data):
  '''
  robust 처리를 위한 함수
  '''

  data = input_data.copy()


  for col in cols:
    scaler = RobustScaler()
    data[col] = scaler.fit_transform(data[col])
  return data

> ## 2.8 PCA

In [ ]:
from sklearn.decomposition import PCA

def pca_do(cols,input_data,compoents=2):
  '''
  pca를 위한 함수
  param:
    cols : pca로 차원 축소 대상 features
    data: 원본 전체 데이터
    compoents : 원하는 차원 수
  
  return :  pca를 추가한 전체 data
  '''

  data = input_data.copy()

  # 차원 축소 하려는 feature 수가 축소할 수보다 작으면 종료
  if len(cols)<=compoents:
    return data

  pca = PCA(n_components= compoents) # 주성분을 몇개로 할지 결정
  printcipalComponents = pca.fit_transform(data[cols])

  #pca 결과 저장을 위한 신규 features 이름 생성
  new_col_name = []
  for i in range(1,compoents+1):
    new_col_name.append('pca'+str(i))


  data[new_col_name] = pd.DataFrame(data=printcipalComponents, columns =new_col_name)

  return data


> ## 2.9 원핫인코딩

In [ ]:
def onehot_do(cols,input_data):
  '''
  원핫인코딩을 위한 함수
  원핫인코딩 대상을 처리하고, 전체 데이터를 return
  '''
  data = input_data.copy()

  return pd.get_dummies(data,columns=cols)

> ## 2.10 기타 처리

>> ### 1) 구간화

In [ ]:
#특정 범위별로 구간화 함수
def binning_do (col,input_data,interval):
  '''
  구간화를 위한 함수
  params 
    col : 구간화 대상 features
    input_data : 원본 데이터
    interval : 구간화를 위한 리스트 
      ex)       [3, 5, 7] 입력시 
                0~2 // 3~4 // 5~6 // 7이상으로 자르고 0,1,2,3... 으로 return 
  '''
  data = input_data.copy()

  data[col] = data.apply(get_label,axis=1,args=[col,interval])

  return data
  

def get_label(df,col,interval):
  ##apply를 위한 함수
  dic = {}

  #딕셔너리 형태로 범위 만들기
  for value,index in enumerate(interval):
    dic[index] = value

  #범위안에 드는지 확인
  for i in dic.keys():
    if df[col] < i:
      return dic[i]
  
  #interval에 포함되지 않는 값은 가장 큰 숫자 return
  return len(dic)



>> ### 2) 값 대치

In [ ]:
def replacement_do (col,input_data,replace_dictionary):
    '''
    원하는 값으로 대치하기 위한 함수
    param:
      col: 대상 feature
      data: 원본데이터
      replace_dictionary : 대치를 위한 조건 사전
                          ex) {'Good':1, 'Standard':2, 'Bad':0}
    '''

    data = input_data.copy()

    data[col] = data.apply(replace,axis=1,args=[col,replace_dictionary])
    return data


def replace(df,col, replace_dictionary):
  #apply를 위한 함수
  dic = replace_dictionary
  for key in dic:
    if df[col] == key:
      return dic[key]


# <span style="background-color:#FAF4C0"><span style='color: black'>3. Preprocessing</span></span>

In [ ]:
# 2.2 drop
#raw 데이터 기준에서 제거
train_df = drop_do(drop_cols, raw_df)

raw_df.shape ,train_df.shape

((100000, 28), (100000, 22))

In [ ]:
#2.3 라벨인코딩
train_df = label_encoding_do(label_cols,train_df)

print('라벨인코딩 전')
print(raw_df[label_cols].head())    
print('\n라벨인코딩 후')
print(train_df[label_cols].head())

라벨인코딩 전
   Customer_ID Payment_of_Min_Amount                 Payment_Behaviour
0         3392                    No   High_spent_Small_value_payments
1         3392                    No    Low_spent_Large_value_payments
2         3392                    No   Low_spent_Medium_value_payments
3         3392                    No    Low_spent_Small_value_payments
4         3392                    No  High_spent_Medium_value_payments

라벨인코딩 후
   Customer_ID  Payment_of_Min_Amount  Payment_Behaviour
0          576                      1                  2
1          576                      1                  3
2          576                      1                  4
3          576                      1                  5
4          576                      1                  1


In [ ]:
#2.4 로그 변환

train_df = log_transform_do(log_scale_cols,train_df)

print('전')
print(raw_df[log_scale_cols].head())    
print('\n후')
print(train_df[log_scale_cols].head())

전
   Monthly_Inhand_Salary
0            1824.843333
1            1824.843333
2            1824.843333
3            1824.843333
4            1824.843333

후
   Monthly_Inhand_Salary
0               7.509249
1               7.509249
2               7.509249
3               7.509249
4               7.509249


In [ ]:
#2.5 sqrt 변환
train_df = sqrt_transform_do(sqrt_scale_cols,train_df)

print('전')
print(raw_df[sqrt_scale_cols].head())    
print('\n후')
print(train_df[sqrt_scale_cols].head())

전
   Outstanding_Debt
0            809.98
1            809.98
2            809.98
3            809.98
4            809.98

후
   Outstanding_Debt
0         28.460148
1         28.460148
2         28.460148
3         28.460148
4         28.460148


In [ ]:
# 2.6 더미 생성

train_df = dummy_make_do(['Num_of_Loan'],train_df,5)
train_df = dummy_make_do(['Changed_Credit_Limit'],train_df,12)
train_df = dummy_make_do(['Credit_History_Age'],train_df,180)

print('전')
print(raw_df[dummy_cols].head())    
print('\n후')
print(train_df[dummy_cols].head())

In [ ]:
#2.7 표준화

train_df = standard_transform_do(standard_cols,train_df)

print('전')
print(raw_df[standard_cols].head())    
print('\n후')
print(train_df[standard_cols].head())

In [ ]:
#2.8 PCA

train_df = pca_do(pca_cols,train_df,2)

print('전')
print(raw_df[pca_cols].head())    

In [ ]:
#2.9 원핫인코딩

train_df = onehot_do(onehot_cols,train_df)


print('전')
print(raw_df[onehot_cols].head())   

In [ ]:
#2.10 기타 처리

##1) 구간화  
##  Age : 특정 숫자 기준으로 구간화
train_df = binning_do('Age',train_df,[20,50])

##2) 값 대치
train_df = replacement_do('Credit_Mix',train_df,{'Bad':0,'Standard':1,'Good':2})
train_df = replacement_do('Credit_Score',train_df,{'Poor':0,'Standard':1,'Good':2})

print('전')
print(raw_df[others_cols].head())    
print('\n후')
print(train_df[others_cols].head())

In [ ]:
## 전처리 결과 확인

pd.set_option('display.max_columns', 30)
# pd.set_option('display.max_rows', 500)


train_df.head(5)


In [ ]:
#전처리 결과 파일 출력 
dir_split= FILE_DIR.split('/')
new_file_name =dir_split[len(dir_split)-1].split('.')[0] +'_preprocess.csv'

dir_split[len(dir_split)-1] = new_file_name
NEW_FILE_DIR = "/".join(dir_split)

#기존 파일디렉터리에 파일명 바꿔서 신규 저장
train_df.to_csv(NEW_FILE_DIR)

# <span style="background-color:#FAF4C0"><span style='color: black'>4. 학습준비</span></span>

> ## 4.1 데이터 split

In [ ]:
# 최종 2개월 valid로 분리
train = train_df.loc[train_df['Month'] <=6]
valid = train_df.loc[train_df['Month'] >6]
train.shape, valid.shape


((75000, 22), (25000, 22))

In [ ]:
# 데이터 셔플
train_shuffled=sklearn.utils.shuffle(train)
valid_shuffled=sklearn.utils.shuffle(valid)

In [ ]:
# train, valid를 각각 x,y 분리

x_train = train_shuffled.drop(['Credit_Score'],axis=1)
y_train = train_shuffled['Credit_Score']

x_valid = valid_shuffled.drop(['Credit_Score'],axis=1)
y_valid = valid_shuffled['Credit_Score']

x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

((75000, 21), (75000,), (25000, 21), (25000,))

In [ ]:
x_train

,Customer_ID,Month,Age,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
77277,9122,6,1,7.239933,4.0,4.0,1.0,0,8.0,7.0,0,0.0,2,27.143323,27.186167,1,0,14.972562,38.577474,1,332.742171
22485,4,6,1,9.039973,3.0,5.0,5.0,0,20.0,14.0,1,6.0,1,37.037819,32.785394,1,2,257.738646,102.488034,5,93.236925
98657,6007,2,1,8.551974,6.0,10.0,29.0,1,33.0,22.0,1,7.0,0,69.508273,32.478309,0,0,187.163787,41.759342,2,261.842180
65058,11822,3,1,8.340937,8.0,7.0,5.0,0,28.0,9.0,1,0.0,1,21.585875,34.181103,1,2,0.000000,74.281953,2,568.417596
42379,3618,4,1,8.719103,2.0,5.0,10.0,0,20.0,7.0,0,0.0,2,29.199658,31.046447,1,1,167.287915,45.436887,1,507.068795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64770,741,3,1,6.441685,5.0,6.0,15.0,0,24.0,16.0,0,2.0,1,17.291038,32.287872,1,1,13.161860,12.233817,5,315.458629
57801,3511,2,1,6.291575,8.0,8.0,18.0,0,59.0,17.0,0,11.0,1,49.422262,24.536619,0,2,13.723019,20.544565,3,278.580264
90857,11610,2,1,9.201088,2.0,4.0,2.0,0,7.0,6.0,0,1.0,2,35.651788,41.522820,1,1,267.529247,65.302556,1,845.663443
21795,2587,4,1,7.813286,4.0,4.0,12.0,0,27.0,12.0,0,1.0,2,27.930449,31.397585,1,0,0.000000,41.012025,5,234.334958


In [ ]:
y_train

77277    1
22485    1
98657    1
65058    1
42379    1
        ..
64770    1
57801    0
90857    2
21795    2
15220    0
Name: Credit_Score, Length: 75000, dtype: int64

In [ ]:
x_valid

,Customer_ID,Month,Age,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
27438,5848,7,2,7.968440,5.0,6.0,19.0,0,8.0,12.0,1,2.0,1,27.795863,36.917234,1,2,164.995629,22.084241,0,469.031974
7278,10086,7,1,8.100467,3.0,4.0,4.0,0,19.0,19.0,0,5.0,1,29.789931,37.337532,1,1,0.000000,65.257258,5,362.411174
64959,5735,8,1,9.221381,5.0,2.0,10.0,0,3.0,12.0,0,6.0,2,18.254041,25.765184,1,1,397.686069,108.724001,2,277.740051
62902,4896,7,0,8.608042,5.0,3.0,18.0,0,20.0,15.0,1,5.0,1,46.407542,38.319145,1,2,152.105906,72.518367,1,500.353886
30486,2538,7,0,8.864427,7.0,5.0,17.0,0,13.0,15.0,0,1.0,1,16.420414,38.945590,1,1,172.346551,55.471201,1,659.193526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62503,1742,8,0,8.572229,9.0,10.0,33.0,1,26.0,19.0,1,8.0,0,54.741209,34.096973,0,2,299.011120,89.405147,1,282.807673
12151,7491,8,1,8.096612,5.0,4.0,2.0,0,9.0,9.0,0,7.0,2,18.267457,34.535877,1,1,44.477372,59.665975,4,344.817774
80166,2490,7,1,7.999625,2.0,5.0,10.0,0,28.0,11.0,0,4.0,2,23.526581,33.561925,1,1,59.880780,65.660468,1,419.981242
52039,3690,8,1,7.283544,9.0,10.0,32.0,0,50.0,15.0,1,11.0,0,47.218534,35.497072,0,2,25.160458,41.361965,1,313.059781


In [ ]:
y_valid

27438    2
7278     0
64959    2
62902    1
30486    1
        ..
62503    1
12151    2
80166    1
52039    0
30710    0
Name: Credit_Score, Length: 25000, dtype: int64